<a href="https://colab.research.google.com/github/tsakailab/sandbox/blob/master/LLM_demo/Fugaku_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

https://www.titech.ac.jp/news/2024/069217

## tokenizer

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model_path = "Fugaku-LLM/Fugaku-LLM-13B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

In [ ]:
tokenizer.tokenize("Zero-shot oral cytology classification via prompt engineering for CLIP models")

['▁Z',
 'ero',
 '-',
 'shot',
 '▁oral',
 '▁cyto',
 'logy',
 '▁classification',
 '▁via',
 '▁pro',
 'mp',
 't',
 '▁engineering',
 '▁for',
 '▁C',
 'LIP',
 '▁models']

In [ ]:
tokenizer.tokenize("視覚的に顕著な細胞の表現学習に基づく口腔細胞画像分類")

['▁',
 '視',
 '覚',
 '的',
 'に',
 '顕',
 '著',
 'な',
 '細胞',
 'の',
 '表',
 '現',
 '学習',
 'に',
 '基づ',
 'く',
 '口腔',
 '細胞',
 '画像',
 '分',
 '類']

In [ ]:
tokenizer.tokenize("顕著性マップを加工した注視による口腔細胞診画像分類の改善")

['▁',
 '顕',
 '著',
 '性',
 'マップ',
 'を',
 '加工',
 'した',
 '注',
 '視',
 'による',
 '口腔',
 '細胞',
 '診',
 '画像',
 '分',
 '類',
 'の',
 '改',
 '善']

## instruct model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "Fugaku-LLM/Fugaku-LLM-13B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto")
model.eval()


config.json:   0%|          | 0.00/859 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/36.2k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

model-00005-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(49152, 5184)
    (wpe): Embedding(2048, 5184)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-39): 40 x GPT2Block(
        (ln_1): LayerNorm((5184,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((5184,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((5184,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=5184, out_features=49152, bias=False)
)

In [ ]:
system_example = "以下は口腔がんの細胞診に関連する技術・用語に関し，情報提供を求める質問です。要求を適切に満たす応答を書きなさい。"
instruction_example = "顕著性マップを加工した注視による口腔細胞診画像分類の改善 という研究テーマはどのような内容が想定できますか？"

prompt = f"{system_example}\n\n### 指示:\n{instruction_example}\n\n### 応答:\n"

input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=128,
    do_sample=True,
    temperature=0.1,
    top_p=1.0,
    repetition_penalty=1.0,
    top_k=0
)
out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)


以下は口腔がんの細胞診に関連する技術・用語に関し，情報提供を求める質問です。要求を適切に満たす応答を書きなさい。

### 指示:
顕著性マップを加工した注視による口腔細胞診画像分類の改善 という研究テーマはどのような内容が想定できますか？

### 応答:
研究課題のひとつに、AIを使って歯科用コーンビームCT画像から口腔がんを自動検出するシステムの開発が考えられます。このシステムは、ディープラーニングアルゴリズムを用いて、コーンビームCT画像から口腔がんを含む異常な組織の特徴を識別するように設計される。具体的には、腫瘍、辺縁隆起、嚢胞、骨折などの異常な組織構造を検出し、医師がより迅速かつ正確に口腔がんを診断できるようにすることを目的としている。

もう一つの研究課題は


## base model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "Fugaku-LLM/Fugaku-LLM-13B"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto")
model.eval()

prompt = "スーパーコンピュータ「富岳」という名称は"

input_ids = tokenizer.encode(prompt,
                             add_special_tokens=False,
                             return_tensors="pt")
tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=128,
    do_sample=True,
    temperature=0.1,
    top_p=1.0,
    repetition_penalty=1.0,
    top_k=0
)
out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin.index.json:   0%|          | 0.00/42.1k [00:00<?, ?B/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/9.78G [00:00<?, ?B/s]

pytorch_model-00002-of-00006.bin:   0%|          | 0.00/9.92G [00:00<?, ?B/s]

pytorch_model-00003-of-00006.bin:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

pytorch_model-00004-of-00006.bin:   0%|          | 0.00/9.92G [00:00<?, ?B/s]

pytorch_model-00005-of-00006.bin:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/4.35G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

スーパーコンピュータ「富岳」という名称は、富士山のように日本の象徴として世界に広く知られ、愛される存在になってほしいとの思いを込めて命名されました。
「富岳」は、「京」の後継機として、理化学研究所と富士通が共同で開発した、スーパーコンピュータです。
「京」の100倍の性能を目指して開発され、2021年6月に共用を開始しました。
「富岳」は、「京」の後継機として、理化学研究所と富士通が共同で開発した、
